# Investigation: Hardware Identification via noise fingerprint exploration
The code to produce the circuits is given below. Thereafter the code to run the ideal circuits is given,then the code to run the circuits on hardware, then the code to extract and store the normalised noise profiles.

In [1]:
import qiskit
from qiskit import QuantumCircuit,  QuantumRegister, ClassicalRegister #Aer, IBMQ,

## Backends
We ibm_torino is Heron r1, ibm_kingston is Heron r2, ibm_pittsburgh is Heron r3.
Would be interesting to see 2 different hardware with same chip.
https://quantum.cloud.ibm.com/computers

In [ ]:
backends = ['ibm_torino','ibm_kingston','ibm_pittsburgh']
#also I think these are the least busy

## Circuits
The circuits are a V shape. They are equivalent to Indentity operations.

In [ ]:
nrs_of_qubits = [4,8,16,32]
circuit_types = ['Cnot','Cnot_X','Swap'] #

In [ ]:
for qubit_nr in nrs_of_qubits:
def make_circuit(nr_qubits,circuit_type):
    qc = QuantumCircuit(nr_qubits)
    qc.h(range(nr_qubits))
    #left side of v
    #right side of v
    for qubit in range(nr_qubits):
        qc.cnot(qubit,qubit+1)
    return qc

def make_right_side(qc,nr_qubits,circuit_type):
    #right side of v
    for qubit in range(nr_qubits):
        match circuit_type:
            case 'Cnot':
                qc.cnot(qubit,qubit+1)
            case 'Cnot_X':
                qc.cnot(qubit,qubit+1)
                qc.x(qubit)
            case 'Swap':
                qc.swap(qubit,qubit+1)

def make_left_side(qc,nr_qubits,circuit_type):
    #right side of v
    for qubit in range(nr_qubits):
        match circuit_type:
            case 'Cnot':
                qc.cnot(qubit,qubit+1)
            case 'Cnot_X':
                qc.cnot(qubit,qubit+1)
                qc.x(qubit)
            case 'Swap':
                qc.swap(qubit,qubit+1)
